# 4.7 基于llama的基因大模型指令微调

### **大模型的指令微调（Instruction Fine-Tuning）**

指令微调是指通过对大语言模型（如 GPT、T5、LLaMA 等）进行微调，使其能够更好地理解和执行人类以指令形式表达的任务。这种技术是大模型适配实际应用和增强用户交互能力的关键手段。

---

### **1. 指令微调的核心概念**

指令微调的目标是通过在包含指令的专用数据集上进行微调，让模型能够：
1. 理解用户的任务需求（以自然语言表达的指令形式）。
2. 根据指令内容生成符合预期的高质量响应。
3. 适应多任务场景，减少特定任务的单独训练需求。

---

### **2. 指令微调的关键特点**

1. **多任务统一**：
   - 不需要针对每个任务单独微调，而是通过指令微调使模型能适应多种任务。
   
2. **自然语言交互**：
   - 用户可以用自然语言指令与模型交互，无需提供特定格式的输入。

3. **泛化能力**：
   - 微调后的模型能够对未见过的任务产生合理的推断和响应。

---

### **3. 数据集的构建与使用**

#### **（1）指令微调数据集的特点**
- 数据通常包含以下三部分：
  1. **指令（Instruction）**：任务描述或问题，例如“将以下文本翻译为法语”。
  2. **输入（Input）**：任务相关的上下文或数据，可以为空。
  3. **输出（Output）**：模型期望生成的结果。

#### **（2）常用指令微调数据集**
- **FLAN**：包含多个 NLP 任务的指令数据集，用于 T5 等模型的微调。
- **OpenAI 提供的指令数据**：如 GPT 系列的 ChatGPT 调优数据集。
- **InstructGPT 数据**：通过人类标注的多任务指令数据，用于模型优化。
- **Self-Instruct**：通过模型自生成指令和回答，进一步扩展训练数据。

#### **（3）构建自己的数据集**
- 如果需要特定领域的指令微调，可以自行构建数据集：
  - 收集任务需求和示例。
  - 设计多样化的指令。
  - 使用专家标注或模型辅助生成高质量答案。

---

### **4. 微调的步骤**

#### **（1）加载基础模型**
从 Hugging Face 或其他框架加载预训练的大语言模型，例如 GPT-2、T5、LLaMA。

#### **（2）准备数据集**
将指令微调数据集格式化为：
```python
{
    "instruction": "Translate the following text to French",
    "input": "Hello, how are you?",
    "output": "Bonjour, comment ça va?"
}
```

#### **（3）定义微调方法**
使用 `Trainer` 或分布式框架（如 DeepSpeed、Accelerate）进行微调。

---

### **5. 示例代码：指令微调实现**

以下是基于 Hugging Face 的指令微调代码示例：

```python
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset

# 1. 加载预训练模型和分词器
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 2. 加载指令微调数据集
# 数据格式应包含 instruction, input, output 字段
dataset = load_dataset("path/to/instruction_dataset")

# 3. 数据预处理
def preprocess_function(example):
    # 将指令和输入拼接成完整的提示
    prompt = example["instruction"]
    if example["input"]:
        prompt += f"\n{example['input']}"
    labels = example["output"]
    tokenized = tokenizer(prompt, truncation=True, max_length=512, padding="max_length")
    with tokenizer.as_target_tokenizer():
        tokenized_labels = tokenizer(labels, truncation=True, max_length=512, padding="max_length")
    tokenized["labels"] = tokenized_labels["input_ids"]
    return tokenized

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 4. 设置训练参数
training_args = TrainingArguments(
    output_dir="./instruction_finetuned_model",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,
)

# 5. 定义 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

# 6. 开始训练
trainer.train()

# 7. 保存模型
model.save_pretrained("./instruction_finetuned_model")
tokenizer.save_pretrained("./instruction_finetuned_model")
```

---

### **6. 指令微调的挑战**

1. **数据质量**：
   - 低质量或噪声数据可能导致模型生成结果不符合指令。

2. **指令覆盖范围**：
   - 数据集指令种类不足会限制模型的泛化能力。

3. **计算资源需求**：
   - 大模型的微调需要高性能 GPU 和大容量存储。

4. **灾难性遗忘**：
   - 微调过程中可能导致模型丧失部分原始能力。

---

### **7. 指令微调的应用场景**

1. **多任务问答**：
   - 适配多任务场景，支持翻译、总结、推理等功能。

2. **特定领域优化**：
   - 在法律、医疗等特定领域的任务指令上进行微调。

3. **用户交互优化**：
   - 提升模型对自然语言指令的理解和响应能力。

4. **开放式对话生成**：
   - 优化模型在对话场景下的表现，例如 ChatGPT 的微调。

---

### **总结**

指令微调通过在特定格式的数据集上进一步训练大模型，使其能够更好地理解和执行用户的自然语言指令。这种方法适合多任务场景，并能提升模型的交互能力和领域适应性。借助高质量的指令数据集和高效的微调技术，大模型在实际应用中的表现可以得到显著提升。

## 持续预训练 VS 指令微调

**大模型的持续预训练**和**指令微调**是两种针对大模型的后续优化策略，虽然它们的目标都是提升模型性能，但在应用场景、方法和效果等方面有明显区别。以下是它们的对比分析：

---

### **1. 概念与目标**

| **特性**               | **持续预训练**                                                                                  | **指令微调**                                                                                              |
|------------------------|-----------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|
| **定义**               | 在通用预训练模型上，使用新的大规模语料（通用或领域特定数据）进行进一步预训练。                      | 在包含指令任务的数据集上对大模型进行微调，以提升模型对人类指令的理解和执行能力。                              |
| **目标**               | 提升模型的通用能力或适应特定领域的语言理解与生成能力。                                              | 提高模型对多任务指令的泛化能力，让模型更好地理解和执行自然语言表达的具体任务。                                |
| **典型应用**           | 领域适配（医学、法律、金融）、性能优化、跨语言适配等。                                              | 多任务问答、开放式对话生成、翻译、推理等需要用户直接交互的场景。                                              |

---

### **2. 数据使用**

| **特性**               | **持续预训练**                                                                                  | **指令微调**                                                                                              |
|------------------------|-----------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|
| **数据类型**           | 通用语料（如新闻、社交媒体文本）或领域特定语料（如 PubMed、法律文档、金融报告）。                    | 任务指令数据集，包括指令（Instruction）、输入（Input）和输出（Output）。                                   |
| **数据构建**           | 通常需要清洗和去重大规模语料数据，避免与原始预训练数据重叠。                                         | 通常由人工标注或模型生成的指令数据构成，例如 FLAN、InstructGPT 数据集。                                     |
| **多样性要求**         | 数据应覆盖尽可能广的领域或目标领域的多种场景，以提升模型在这些场景的表现。                                | 数据需要覆盖多种任务类型（如翻译、分类、摘要）和丰富的指令表达形式，以提高模型对多任务的适配能力。                 |

---

### **3. 方法与技术**

| **特性**               | **持续预训练**                                                                                  | **指令微调**                                                                                              |
|------------------------|-----------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|
| **主要技术**           | 继续使用自监督学习目标（如语言建模、掩码预测）进行训练。                                            | 使用监督学习，通常以任务输入和目标输出对为数据，通过微调适配特定任务需求。                                  |
| **模型调整**           | - 可选择全量参数更新或冻结部分参数。<br>- 可结合参数高效微调技术（如 LoRA、Adapter）。                  | - 通常使用监督训练方式，可能结合参数高效微调技术（如 LoRA）。                                               |
| **学习率**             | 通常使用较小的学习率（如 `1e-5` 或更小），以防止破坏原始权重。                                         | 同样使用较小的学习率，但任务指令微调可能需要更高的关注任务特定的标签对准。                                     |

---

### **4. 模型能力与效果**

| **特性**               | **持续预训练**                                                                                  | **指令微调**                                                                                              |
|------------------------|-----------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|
| **提升的能力**         | - 对领域特定语言模式和知识的适配性提升显著。<br>- 对未见过的通用场景生成能力增强（扩展模型知识广度）。       | - 显著提升模型对指令理解的能力，尤其是自然语言表达的任务需求。<br>- 对多任务和零样本任务的泛化能力有较大提升。  |
| **局限性**             | - 对具体任务的直接适配能力较弱，可能需要额外的任务微调。<br>- 数据选择不当可能导致灾难性遗忘。                 | - 依赖高质量的指令数据集，数据质量不高会导致模型生成结果不稳定。<br>- 对通用能力的提升有限。                    |

---

### **5. 应用场景与示例**

| **特性**               | **持续预训练**                                                                                  | **指令微调**                                                                                              |
|------------------------|-----------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|
| **典型应用场景**       | - 医学文献总结（通过 PubMed 语料持续预训练）。<br>- 法律条文分析（通过法律文档进一步训练）。<br>- 增强多语言生成能力（跨语言语料）。 | - ChatGPT 的多任务对话生成。<br>- 翻译、摘要、问答等用户交互任务的泛化处理。                                 |
| **实际示例**           | - BioBERT：在 BERT 基础上使用生物医学语料持续预训练的模型。<br>- FinBERT：针对金融领域持续预训练的语言模型。 | - InstructGPT：在 GPT-3 基础上进行指令微调，用于多任务用户交互。<br>- FLAN-T5：通过 FLAN 数据集进行指令微调。 |

---

### **6. 持续预训练与指令微调的结合**

持续预训练和指令微调可以结合使用，形成一个从领域适配到任务适配的完整流程：
1. **持续预训练**：
   - 先在领域特定数据（如医学、法律、金融语料）上进行持续预训练，获取领域知识。
2. **指令微调**：
   - 再利用多任务指令数据集对模型微调，使其能够高效执行领域内的多样化任务。

这种结合方式特别适用于需要领域知识和任务适配的场景，例如医学问答系统或金融文本分析。

---

### **总结**

| **维度**               | **持续预训练**                      | **指令微调**                     |
|------------------------|-------------------------------------|----------------------------------|
| **目标**               | 增强通用能力或适配特定领域。          | 提升对任务指令的理解和执行能力。    |
| **数据集**             | 通用或领域语料。                    | 指令数据集，包含输入和输出对。      |
| **方法**               | 自监督学习，扩展语言建模能力。         | 监督学习，强化任务适配能力。        |
| **适用场景**           | 领域特定任务（如医学、法律）。         | 多任务交互（如问答、对话生成）。     |
| **局限性**             | 对具体任务适配较弱。                 | 通用能力提升有限，依赖数据质量。     |

两者各有侧重，且在许多场景下可以结合使用，形成一个强大的任务和领域适配框架。

## 本节任务
本节任务是基于上一节预训练的llama生物大模型。对一些生物学任务进行微调，包含了多个不同类型的分类问题和多序列交换问题。具体可见sft_data下的数据。

## 代码运行

```

#微调
./run_sft.sh

运行时间约3小时

#合并模型
./merge_sft_model.sh

```

## 模型验证

In [1]:
import subprocess
import os
# 设置环境变量, autodl一般区域
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
from transformers import AutoTokenizer, AutoConfig,AutoModel
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import  AutoConfig, AutoModelForCausalLM,LlamaForCausalLM,LlamaTokenizer
from tokenizers import Tokenizer
from datasets import load_dataset

In [3]:
from datasets import load_dataset
dna_ft_dataset = load_dataset('json', data_files='val_data.json')
dna_ft_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 19839
    })
})

In [4]:
data = dna_ft_dataset["train"].train_test_split(train_size=0.1, seed=42)
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1983
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 17856
    })
})

In [5]:
tokenizer = LlamaTokenizer.from_pretrained("dnahlm-llama-7b-sft-v0") #dnagpt/dnahlm-llama-7b-sft-v0
tokenizer.pad_token = tokenizer.eos_token

In [6]:
model = LlamaForCausalLM.from_pretrained("dnahlm-llama-7b-sft-v0") #continue pretrain
model

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(91644, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      

In [7]:
#构建提示词
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text + "\n\n### Response:\n"

#构建提示词
def build_prompt(entry):

    input_data = format_input(entry)

    desired_response = entry['output']

    return input_data + desired_response


In [8]:
example = data["test"][0]
example

{'instruction': 'Determine core promoter detection of following dna sequence, The result will be one of the following: Non-promoter, promoter.',
 'input': 'CCGTGCGACCGGAAGTGGGGCGGCGACCCCGGAAGTCCCCGCCGGGTGCAGCTTGGTCGGTTCGATCGCC',
 'output': 'promoter'}

In [9]:
prompt = build_prompt(example)
print(prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Determine core promoter detection of following dna sequence, The result will be one of the following: Non-promoter, promoter.

### Input:
CCGTGCGACCGGAAGTGGGGCGGCGACCCCGGAAGTCCCCGCCGGGTGCAGCTTGGTCGGTTCGATCGCC

### Response:
promoter


In [10]:
tokenizer.tokenize(prompt)

['▁Below',
 '▁is',
 '▁an',
 '▁instruction',
 '▁that',
 '▁describes',
 '▁a',
 '▁task',
 '.',
 '▁Write',
 '▁a',
 '▁response',
 '▁that',
 '▁appropri',
 'ately',
 '▁comple',
 'tes',
 '▁the',
 '▁request',
 '.',
 '<0x0A>',
 '<0x0A>',
 '##',
 '#',
 '▁Inst',
 'ruction',
 ':',
 '<0x0A>',
 'Det',
 'erm',
 'ine',
 '▁core',
 '▁prom',
 'oter',
 '▁detection',
 '▁of',
 '▁following',
 '▁d',
 'na',
 '▁sequence',
 ',',
 '▁The',
 '▁result',
 '▁will',
 '▁be',
 '▁one',
 '▁of',
 '▁the',
 '▁following',
 ':',
 '▁Non',
 '-',
 'prom',
 'oter',
 ',',
 '▁prom',
 'oter',
 '.',
 '<0x0A>',
 '<0x0A>',
 '##',
 '#',
 '▁Input',
 ':',
 '<0x0A>',
 'CCG',
 'TGCG',
 'ACCGG',
 'AAG',
 'TGGGGC',
 'GGCG',
 'ACCCCGG',
 'AAG',
 'TCCCC',
 'GCCGGG',
 'TGCAGC',
 'TTGG',
 'TCGG',
 'TTCG',
 'ATCGCC',
 '<0x0A>',
 '<0x0A>',
 '##',
 '#',
 '▁Response',
 ':',
 '<0x0A>',
 'prom',
 'oter']

In [11]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=1000):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
          # return_attention_mask=True,
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    #max_length=max_output_tokens,
    max_new_tokens=8,
    temperature=0.01  # 控制生成的多样性
  )

  # Decode
  generated_text_with_prompt = tokenizer.decode(generated_tokens_with_prompt[0], skip_special_tokens=True)
  generated_text_answer = generated_text_with_prompt[len(text):]


  return generated_text_answer

# 如果需要进一步清理
def clean_generated_text(text):
    # 去除 'Ġ' 符号并替换为空格
    text = text.replace('Ġ', ' ')
    # 去除多余的空格
    text = ' '.join(text.split())
    return text

In [12]:
input_text = format_input(data["test"][0])

print("input (test):", input_text)

print("real answer:", data["test"][0]["output"])

print("--------------------------\n")

print("model's answer: \n")
print(inference(input_text, model, tokenizer))

input (test): Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Determine core promoter detection of following dna sequence, The result will be one of the following: Non-promoter, promoter.

### Input:
CCGTGCGACCGGAAGTGGGGCGGCGACCCCGGAAGTCCCCGCCGGGTGCAGCTTGGTCGGTTCGATCGCC

### Response:

real answer: promoter
--------------------------

model's answer: 



/root/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


promoter


In [ ]:
test_data = data["test"].shuffle(seed=199).select(range(100))

data_list = []

for entry in test_data:
    input_text = format_input(entry)
    #print(input_text)
    response_text = inference(input_text, model, tokenizer)
    #print(response_text)
    data = {
        "instruction":entry["instruction"],
         "input":entry["input"],
         "output":entry["output"],
        "model_response":response_text
    }

    data_list.append(data)

In [ ]:
import json

# 定义输出文件路径
output_file = 'llama-sft-2.json'

# 将 Dataset 对象导出为 JSON 文件
# test_data.to_json(output_file)
with open(output_file, "w") as file:
    json.dump(data_list, file, indent=4)  # "indent" for pretty-printing



In [16]:
import json
from tqdm import tqdm



with open(output_file, "r") as file:
    test_data = json.load(file)

all_num = len(test_data)
right_sum = 0
same_sum = 0
for item in test_data:
    output = item["output"]
    #output = " ".join(tokenizer.tokenize(output))
    model_response = item["model_response"]

    print(output,"||||||||||||", model_response)

    if model_response == output: #same it
        same_sum = same_sum + 1
        
    if output.find("Non")==-1: # no Non
        if model_response.find(output)!=-1 and model_response.find("Non")==-1: #find it, but no Non
            right_sum = right_sum + 1
    else:
        if model_response.find(output)!=-1: #find it
            right_sum = right_sum + 1


print("presicion", right_sum/all_num, "same", same_sum/all_num)


Donor Sites |||||||||||| Non-Splice Sites
promoter |||||||||||| promoter
promoter |||||||||||| promoter
promoter ||||||||||||  Non-promoter
promoter |||||||||||| promoter
Donor Sites ||||||||||||  Non-Splice Sites
promoter |||||||||||| promoter
promoter ||||||||||||  Non-promoter
Non-promoter |||||||||||| promoter
Non-promoter ||||||||||||  Non-promoter
Donor Sites ||||||||||||  Donor Sites
Non-promoter ||||||||||||  Non-promoter
Non-promoter ||||||||||||  Non-promoter
Non-promoter |||||||||||| promoter
promoter |||||||||||| promoter
promoter |||||||||||| promoter
Donor Sites |||||||||||| Splice Sites
Background Sequences ||||||||||||  Background Sequences
Non-promoter ||||||||||||  Non-promoter
Non-promoter ||||||||||||  Non-promoter
promoter ||||||||||||  Non-promoter
promoter |||||||||||| promoter
promoter |||||||||||| promoter
promoter ||||||||||||  Non-promoter
promoter |||||||||||| promoter
promoter |||||||||||| promoter
Non-promoter ||||||||||||  Non-promoter
Non-Splice Sites ||